In [2]:
# Import base libraries
from numpy import *
from numpy.linalg import multi_dot
import pandas as pd
import yfinance as yf

# Import cufflinks
import cufflinks as cf
cf.set_config_file(
    offline=True, 
    dimensions=((1000,600)),
    theme= 'henanigans')

# Import plotly express for EF plot
import plotly.express as px
px.defaults.template = "plotly_dark"
px.defaults.width = 1000
px.defaults.height = 600

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Nasdaq-listed stocklist
symbols = ['AMD', 'CSCO', 'INTC', 'INTU', 'NVDA']

# Number of assets
numofasset = len(symbols)

# Number of portfolio for optimization
numofportfolio = 5000

In [17]:
yf.download(symbols, start='2017-01-01', end='2021-12-31', progress=False)['Adj Close']


5 Failed downloads:
- INTU: No data found for this date range, symbol may be delisted
- INTC: No data found for this date range, symbol may be delisted
- NVDA: No data found for this date range, symbol may be delisted
- CSCO: No data found for this date range, symbol may be delisted
- AMD: No data found for this date range, symbol may be delisted


,AMD,CSCO,INTC,INTU,NVDA
Date,,,,,


In [6]:
# Fetch data from yahoo finance for last six years
# nasdaqstocks = yf.download(symbols, start='2017-01-01', end='2021-12-31', progress=False)['Adj Close']
# nasdaqstocks.to_csv('data/nasdaqstocks.csv')

# Load locally stored data
df = pd.read_csv('nasdaqstocks.csv', index_col=0, parse_dates=True)
# Verify the output
df

,AMD,CSCO,INTC,INTU,NVDA
Date,,,,,
2017-01-03,11.430000,25.985624,31.973000,110.028526,25.166451
2017-01-04,11.430000,25.831152,31.807018,110.249046,25.753607
2017-01-05,11.240000,25.891224,31.754604,110.929825,25.099840
2017-01-06,11.320000,25.942715,31.868174,112.381981,25.435360
2017-01-09,11.490000,25.899811,31.981735,112.045395,26.466593
...,...,...,...,...,...
2021-12-23,146.139999,61.915154,50.922092,634.972839,296.349487
2021-12-27,154.360001,63.048477,51.547329,652.013062,309.397308
2021-12-28,153.149994,63.157833,51.368690,649.026550,303.168335


In [7]:
# Plot price history
df['2021':].normalize().iplot(kind='line', title='Normalized Price Plot')

In [8]:
# Calculate returns 
returns = df.pct_change().fillna(0)
returns.tail()

,AMD,CSCO,INTC,INTU,NVDA
Date,,,,,
2021-12-23,0.015707,0.012189,0.006671,0.006412,0.008163
2021-12-27,0.056247,0.018304,0.012278,0.026836,0.044028
2021-12-28,-0.007839,0.001734,-0.003466,-0.004580,-0.020133
2021-12-29,-0.031929,0.006768,0.001352,-0.002693,-0.010586
2021-12-30,-0.020977,-0.005316,-0.001736,-0.007206,-0.013833


In [9]:
# Plot annualized return and volatility
pd.DataFrame({
    'Annualized Return' : round(returns.mean() * 252 * 100,2),
    'Annualized Volatility': round(returns.std()*sqrt(252) * 100,2)
}).iplot(kind='bar', title='Annualized Return & Volatility (%)', shared_xaxes = True, subplots=True)

In [20]:
# Define Weights for Equal weighted portfolio
wts = numofasset * [1./numofasset]
wts

[0.2, 0.2, 0.2, 0.2, 0.2]

In [25]:
# Reshape
wts = numofasset * [1./numofasset]
wts = array(wts)[:,newaxis]
wts

array([[0.2],
       [0.2],
       [0.2],
       [0.2],
       [0.2]])

In [26]:
wts.shape

(5, 1)

In [27]:
# Derive returns
ret = array(returns.mean() * 252)[:,newaxis]      
ret

array([[0.66721301],
       [0.21500207],
       [0.15552518],
       [0.40136363],
       [0.60531437]])

In [28]:
def portfolio_simulation(returns):

    # Initialize the lists
    rets = []; vols = []; wts = []

    # Simulate 5,000 portfolios
    for i in range (numofportfolio):
        
        # Generate random weights
        weights = random.random(numofasset)[:, newaxis]
        
        # Set weights such that sum of weights equals 1
        weights /= sum(weights)
        
        # Portfolio statistics
        rets.append(weights.T @ array(returns.mean() * 252)[:, newaxis])        
        vols.append(sqrt(multi_dot([weights.T, returns.cov()*252, weights])))
        wts.append(weights.flatten())

    # Create a dataframe for analysis
    portdf = 100*pd.DataFrame({
        'port_rets': array(rets).flatten(),
        'port_vols': array(vols).flatten(),
        'weights': list(array(wts))
        })
    
    portdf['sharpe_ratio'] = portdf['port_rets'] / portdf['port_vols']

    return round(portdf,2)

In [29]:
# Create a dataframe for analysis
temp = portfolio_simulation(returns)
temp.head()

,port_rets,port_vols,weights,sharpe_ratio
0,32.09,27.93,"[11.892657112932655, 51.25222746893543, 19.633...",1.15
1,39.62,31.99,"[23.061921538021547, 16.68007236315871, 31.369...",1.24
2,41.64,32.23,"[15.943664482758699, 15.979009061835411, 24.73...",1.29
3,48.50,35.01,"[38.35302373845929, 25.9626050572183, 3.090453...",1.39
4,53.35,39.27,"[7.711114426318298, 2.281438262435384, 7.98795...",1.36


In [30]:
# Get the max sharpe portfolio stats
temp.iloc[temp.sharpe_ratio.idxmax()]

port_rets                                                   46.88
port_vols                                                   31.54
weights         [20.752842094286716, 11.332511170747965, 0.439...
sharpe_ratio                                                 1.49
Name: 2378, dtype: object

In [31]:
# Verify the above result
temp.describe().T

,count,mean,std,min,25%,50%,75%,max
port_rets,5000.0,40.740220,5.848307,20.16,36.99,40.69,44.50,61.07
port_vols,5000.0,31.436644,2.911133,25.28,29.40,31.09,33.11,48.95
sharpe_ratio,5000.0,1.291782,0.101040,0.67,1.24,1.31,1.36,1.49


In [32]:
# Max sharpe ratio portfolio weights
msrpwts = temp['weights'][temp['sharpe_ratio'].idxmax()]

# Allocation to achieve max sharpe ratio portfolio
dict(zip(symbols, around(msrpwts,2)))

{'AMD': 20.75, 'CSCO': 11.33, 'INTC': 0.44, 'INTU': 50.6, 'NVDA': 16.87}

In [33]:
# Plot simulated portfolio
fig = px.scatter(
    temp, x='port_vols', y='port_rets', color='sharpe_ratio', 
    labels={'port_vols': 'Expected Volatility', 'port_rets': 'Expected Return','sharpe_ratio': 'Sharpe Ratio'},
    title="Monte Carlo Simulated Portfolio"
     ).update_traces(mode='markers', marker=dict(symbol='cross'))

# Plot max sharpe 
fig.add_scatter(
    mode='markers', 
    x=[temp.iloc[temp.sharpe_ratio.idxmax()]['port_vols']], 
    y=[temp.iloc[temp.sharpe_ratio.idxmax()]['port_rets']], 
    marker=dict(color='RoyalBlue', size=20, symbol='star'),
    name = 'Max Sharpe'
).update(layout_showlegend=False)

# Show spikes
fig.update_xaxes(showspikes=True)
fig.update_yaxes(showspikes=True)
fig.show()